In [ ]:
import nltk
from Levenshtein import distance
import pandas as pd
import json
from tqdm import tqdm
from os.path import join, isdir, isfile, abspath, dirname, splitext, basename, split
from derive_conceptualspace.util.mpl_tools import show_hist
from collections import Counter

BOOK_BASE = "/home/chris/Documents/UNI_neu/Masterarbeit/OTHER/study_behavior_analysis/src/"

# Looking at the DDC-Classification of the course Dataset

In [ ]:
path = "/home/chris/Documents/UNI_neu/Masterarbeit/OTHER/study_behavior_analysis/EducationalResource-2022-01-20.csv"

df = pd.read_csv(path)
assert all(i.startswith('"') and i.endswith('"') and i[1:-1].isnumeric() for i in df["ddc_code"] if not pd.isna(i))
df["ddc_code"] = df["ddc_code"].str[1:-1]#.astype(pd.Int64Dtype())
df = df.drop(columns=["identifier", "contributor", "creator", "coverage", "date", "rights", "relation"])
df["description"] = df["description"].str.strip().str.replace("\r\n", "\n") 
df["description"] = df["description"].str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n")
df["description"] = df["description"].str.replace("\n", " ") 
df["description"] = df["description"].str.strip()
df["title"] = df["title"].str.strip()
display(df.describe())
print("\n\n")
display(df.head())
print("\n\n")
display(df["origin"].value_counts())

In [ ]:
print(df["type"].unique())
df["type"] = df["type"].str.replace("['SIP']", "SIP", regex=False).str.replace("['udemy', 'mooc']", "udemy_mooc", regex=False).str.replace("['OER']", "OER", regex=False)
print(df["format"].unique())
df["format"] = df["format"].str.replace("['CRS']", "CRS", regex=False)
df = df.set_index("id")
df = df.dropna(subset=["title"])
df.head()

In [ ]:
# df = df[df["description"].str.count(" ") >= 9]
# df.describe()

In [ ]:
# display(df["publisher"].value_counts())
# seldom_publishers = [k for k,v in df["publisher"].value_counts().items() if v <= 20]
# display(df[df["publisher"].isin(seldom_publishers)])
# print(df[df["publisher"].isin(seldom_publishers)]["relation"][1129]) #<- links sind tot
# print(df[df["publisher"].isin(seldom_publishers)]["source"][1129])   #<- links sind tot

# #wir sehen, die sind alle of type OER. 
# #Interessante Frage wäre noch was Relation ist

In [ ]:
# df = df[~df["publisher"].isin(seldom_publishers)]
# df = df.set_index("id")
# df = df.drop(columns=["relation", "source"])
# df

## Looking at course data

In [ ]:
# df = pd.read_csv(join(BOOK_BASE, "data/course_data/db_dump_new", "institute_dump.csv"))
# print(len(df))
# df.head()

In [ ]:
# df = pd.read_csv(join(BOOK_BASE, "data/course_data/db_dump_new", "lecturer_dump.csv"))
# print(len(df))
# df.head()

In [ ]:
cdn = pd.read_csv(join(BOOK_BASE, "data/course_data/db_dump_new", "course_dump_new.csv"))
print(len(cdn))
cdn = cdn.drop(columns=["course_origin_id", "place", "start_time", "end_time", "end_semester", "date", "TF_IDF_scores", "origin_id"])
#display(cdn.head())
assert all(i.startswith('"\\"') and i.endswith('\\""') and i[3:-3].isnumeric() for i in cdn["ddc_code"] if not pd.isna(i))
cdn["ddc_code"] = cdn["ddc_code"].str[3:-3]
cdn = cdn.set_index("id")
cdn = cdn.dropna(subset=["title"])
cdn["description"] = cdn["description"].str.strip().str.replace("\r\n", "\n")
cdn["description"] = cdn["description"].str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n")
cdn["description"] = cdn["description"].str.replace("\n", " ")
cdn["description"] = cdn["description"].str.strip()
cdn["title"] = cdn["title"].str.strip()
cdn.head()

In [ ]:
edu = pd.read_csv(join(BOOK_BASE, "data/course_data/db_dump_new", "eduresource_dump.csv"))
print(len(edu))
print("#Entries for 'contributor' column:", len(edu[(~edu["contributor"].isna()) & (~edu["contributor"].isin([[], "[]"]))]))
print("#Entries for 'creator' column:", len(edu[(~edu["creator"].isna()) & (~edu["creator"].isin([[], "[]"]))]))
# edu.head()
edu = edu.drop(columns=["TF_IDF_scores", "contributor", "date", "relation", "rights", "origin_id"])
display(edu["format"].unique())
edu["format"] = edu["format"].str.replace('["udemy", "mooc"]', "udemy_mooc", regex=False).str.replace('["CRS"]', "CRS", regex=False).str.replace('"video/mp4"', "mp4", regex=False).str.replace('[]', "", regex=False)
display(edu["type"].unique())
edu["type"] = edu["type"].str.replace('["SIP"]', "SIP", regex=False).str.replace('["udemy", "mooc"]', "udemy_mooc", regex=False).str.replace('["video", "OER"]', "video_OER", regex=False).str.replace('["WEB"]', "web", regex=False)
display(edu["type"].unique())
edu = edu.set_index("id")
assert all(i.startswith('"\\"') and i.endswith('\\""') and i[3:-3].isnumeric() for i in edu["ddc_code"] if not pd.isna(i))
edu["ddc_code"] = edu["ddc_code"].str[3:-3]
edu = edu.dropna(subset=["title"])
edu["description"] = edu["description"].str.strip().str.replace("\r\n", "\n")
edu["description"] = edu["description"].str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n")
edu["description"] = edu["description"].str.replace("\n", " ")
edu["description"] = edu["description"].str.strip()
edu["title"] = edu["title"].str.strip()
edu.head()

## Merging them

In [ ]:
print(len(df))
print(len(cdn))
print(len(edu))
display(df.head(2))
display(cdn.head(2))
display(edu.head(2))
print(list(df.dtypes))
print(list(cdn.dtypes))
print(list(edu.dtypes))

### merging. First the dumps `df` and `cdn`

In [ ]:
cdn["type"] = "SIP"

In [ ]:
print(len(set(df["title"])), len(set(cdn["title"])), len(set(df["title"])&set(cdn["title"])), len(set(df["title"])|set(cdn["title"])))

In [ ]:
df["description"] = df["description"].str.replace("\n", ". ")
cdn["description"] = cdn["description"].str.replace("\n", ". ")
tmp = df[df["type"] == "SIP"].merge(cdn, on="title", how="left", suffixes=("_df", "_cdn"))
#TODO tmp += df[df["type"] != "SIP"]
tmp

In [ ]:
tmp = tmp.drop(columns=["origin", "type_df", "type_cdn"])
tmp["type"] = "SIP"
print("Courses that have a different DDC in the two joined dataframes:")
display(tmp[(~tmp["ddc_code_df"].isna()) & (~tmp["ddc_code_cdn"].isna()) & (tmp["ddc_code_df"] != tmp["ddc_code_cdn"])])

In [ ]:
tmp["ddc_code"] = [i[1]["ddc_code_cdn"] if not pd.isna(i[1]["ddc_code_cdn"]) else i[1]["ddc_code_df"] for i in tmp.iterrows()] #so this may lead to a wrong ddc for TWO samples, that's okay.
tmp = tmp.drop(columns=["ddc_code_cdn", "ddc_code_df"])
tmp.head()

In [ ]:
print(f'equal descriptions for both: {len(tmp[tmp["description_df"]==tmp["description_cdn"]])}')
#display(tmp[tmp["description_df"]!=tmp["description_cdn"]][["title", "description_df", "description_cdn"]])
different_descriptions_mask = (((~tmp["description_df"].isna())&(~tmp["description_cdn"].isna())) & (tmp["description_df"].str.lower()!=tmp["description_cdn"].str.lower()))
with pd.option_context('display.max_rows', 5, 'display.max_columns', 20, 'display.expand_frame_repr', False, 'display.max_colwidth', 10000, 'display.float_format', '{:.4f}'.format):
    display(tmp[different_descriptions_mask][["title", "description_df", "description_cdn"]])
different_descriptions = tmp[different_descriptions_mask][["title", "description_df", "description_cdn"]]

In [ ]:
#Left to do: 
# * tmp += df[df["type"] != "SIP"] (add the non-courses from df)
# * das koomplette `edu` ebenfalls reinmergen
# * die 8432 with equal descriptions einfach die eine übernehmen
# * noch nen bisschen smarter mergen (bei einigen ist nur die Jahreszahl anders, bei einigen steht halt nur "online" dabei, etc) (sent_tokenize und einfach jeden satz 1 mal übernehmen?)
# * nachdem edu reingemergt ist die mit komplett leeren descriptions löschen
# * bei zeilen mit gleichen titeln auch die descriptions concatenaten
# * merge with the siddata2021 dataset at least because I need the course number from there
# * doch die origin-column behalten, vielleicht erkenn ich da die originalen siddata2021 kurse dran
# * merged_all["coverage"] muss an die finale description angehängt werden
# --> mit den description_df und description_cdn einfach genauso umgehen und die zu einer spalten machen und dann zeilenweise den sent_tokenize kram machen
# Also: 1) die non-courses von df reinmergen 2) edu reinmergen, 3) da wo description_df = description_cdn (oder != description_edu) extra zeilen draus machen 4) dafür dann sent_tokenize etc
#PS: tried other way round but this makes more sense: FIRST make separate rows for where description_cdn and description_df differ.

In [ ]:
tmp = tmp.reindex(sorted(tmp.columns), axis=1)
tmp.head()

In [ ]:
nonsip = df[df["type"] != "SIP"].rename(columns={"description": "description_df"})
nonsip = nonsip.reset_index().drop(columns=["id", "origin"])
nonsip["description_cdn"] = pd.NA
nonsip["start_semester"] = pd.NA
nonsip["url"] = pd.NA
nonsip = nonsip.reindex(sorted(nonsip.columns), axis=1)
print("tmp-len", len(tmp), "nonsip-len", len(nonsip))
tmp = pd.concat([tmp, nonsip])
print("merged-len", len(tmp))
display(tmp)

In [ ]:
new_merged = tmp[((tmp["description_cdn"].isna())^(tmp["description_df"].isna()))]  #ok so first only those with only one description
print(len(new_merged))
new_merged = pd.concat([new_merged, tmp[tmp["description_cdn"]==tmp["description_df"]]]) #then those where the description is equal
print(len(new_merged)) 
new_merged = pd.concat([new_merged, tmp[(tmp["description_cdn"].isna()) & (tmp["description_df"].isna())]]) #then those who don't have any description at all
print(len(new_merged)) 
#thing that's left: those where there is a description for both and it's different
different_desc_mask = (~tmp["description_cdn"].isna()) & (~tmp["description_df"].isna()) & (tmp["description_cdn"]!=tmp["description_df"])
assert len(new_merged)+len(tmp[different_desc_mask]) == len(tmp) #ensure these 4 types are actually all
new_merged["description"] = [i["description_cdn"] if not pd.isna(i["description_cdn"]) else i["description_df"] for num, i in new_merged.iterrows()]
new_merged = new_merged.drop(columns=["description_cdn", "description_df"])
new_merged

In [ ]:
#display(tmp[different_desc_mask].head())
doublings = tmp[different_desc_mask]
doublings["description"] = [[row["description_cdn"], row["description_df"]] for num, row in doublings.iterrows()]
doublings = doublings.drop(columns=["description_cdn", "description_df"])
doublings = doublings.explode("description")

new_merged = pd.concat([new_merged, doublings])
new_merged = new_merged.reindex(sorted(new_merged.columns), axis=1)
print(len(new_merged[new_merged["description"].isna()]))
new_merged

In [ ]:
#Left to do: 
# * das koomplette `edu` ebenfalls reinmergen und exploden
# * nachdem edu reingemergt ist die mit komplett leeren descriptions löschen
# * danach grouped-by-title die descriptions smart mergen
#     * bei einigen ist nur die Jahreszahl anders, bei einigen steht halt nur "online" dabei, etc 
#     * sent_tokenize und einfach jeden satz 1 mal übernehmen?
#     * auf jeden Fall ungleiches concatenaten, und dann pro kurs die maximale nichtdoppelte beschreibung haben
# * merge with the siddata2021 dataset at least because I need the course number from there
#    * doch die origin-column behalten, vielleicht erkenn ich da die originalen siddata2021 kurse dran
# * new_merged["coverage"] muss an die finale description angehängt werden

#### merging with edu

In [ ]:
edu = edu.reindex(sorted(edu.columns), axis=1)
edu = edu.reset_index().drop(columns=["id"])
display(edu.head())
print(len(edu))
display(new_merged.head())
print(len(new_merged))

In [ ]:
merged_all = new_merged.merge(edu, on="title", how="left", suffixes=("_dfcdn", "_edu"))
#merged_all = merged_all.rename(columns={"description": "description_edu"})
merged_all = merged_all.reindex(sorted(merged_all.columns), axis=1)
print(len(merged_all))
merged_all = merged_all.drop(columns=["identifier"])
merged_all.head()

In [ ]:
#now I can delete those that don't have any description in either of the 3 csvs
merged_all = merged_all[((~merged_all["description_dfcdn"].isna())|(~merged_all["description_edu"].isna()))]
print(len(merged_all))

merged_all["description_dfcdn"] = merged_all["description_dfcdn"].str.replace(r'<.*?>', '', regex=True)
merged_all["description_edu"] = merged_all["description_edu"].str.replace(r'<.*?>', '', regex=True)

In [ ]:
# now they may still differ in description, but also in type, coverage, creator, format, ddc ...
merged_all.head()

In [ ]:
new_all = merged_all[((merged_all["description_dfcdn"].isna())^(merged_all["description_edu"].isna()))]  #ok so first only those with only one description
print(len(new_all))
new_all = pd.concat([new_all, merged_all[merged_all["description_dfcdn"]==merged_all["description_edu"]]]) #then those where the description is equal
print(len(new_all)) 
#thing that's left: those where there is a description for both and it's different
different_desc_mask = (~merged_all["description_dfcdn"].isna()) & (~merged_all["description_edu"].isna()) & (merged_all["description_dfcdn"]!=merged_all["description_edu"])
assert len(new_all)+len(merged_all[different_desc_mask]) == len(merged_all) #ensure these 3 types are actually all (there are no full NA-rows anymore)
#now we need to take the other information from either edu or dfdcn, depending on where it came from
for col in ["ddc_code", "format", "language", "publisher", "source", "type"]:
    print(col)
    with pd.option_context('display.max_rows', 5, 'display.max_columns', 25, 'display.expand_frame_repr', False, 'display.max_colwidth', 20, 'display.float_format', '{:.4f}'.format):
        display(new_all[(new_all[f"{col}_dfcdn"] != new_all[f"{col}_edu"]) & (~new_all[f"{col}_dfcdn"].isna()) & (~new_all[f"{col}_edu"].isna())])
    #new_all[col] = [i[f"{col}_dfcdn"] if not pd.isna(i["description_dfcdn"]) else i[f"{col}_edu"] for num, i in new_all.iterrows()]
    #new_all = new_all.drop(columns=[f"{col}_dfcdn", f"{col}_edu"])
# with pd.option_context('display.max_rows', 5, 'display.max_columns', 25, 'display.expand_frame_repr', False, 'display.max_colwidth', 20, 'display.float_format', '{:.4f}'.format):
#     display(new_all.head())
#new_all["description"] = [i["description_dfcdn"] if not pd.isna(i["description_dfcdn"]) else i["description_edu"] for num, i in new_all.iterrows()]
#new_all = new_all.drop(columns=["description_dfcdn", "description_edu"])
#new_all = new_all.reindex(sorted(new_all.columns), axis=1)
#new_all.head()

# You know what, maybe I just explode them all.

In [ ]:
# So, I will make one row per description and for the other columns (ddc_code, language, ...) I'll just concatenate dfcdn and edu, and then I'll groupby names and concatenate EVERYTHING
with pd.option_context('display.max_rows', 5, 'display.max_columns', 25, 'display.expand_frame_repr', False, 'display.max_colwidth', 20, 'display.float_format', '{:.4f}'.format):
    display(merged_all)

In [ ]:
for col in ["ddc_code", "format", "language", "publisher", "source", "type", "subject"]:
    merged_all[col] = [tuple([row[f"{col}_dfcdn"], row[f"{col}_edu"]]) for num, row in merged_all.iterrows()]
    merged_all = merged_all.drop(columns=[f"{col}_dfcdn", f"{col}_edu"])
    
merged_all = merged_all.reindex(sorted(merged_all.columns), axis=1)
with pd.option_context('display.max_rows', 5, 'display.max_columns', 25, 'display.expand_frame_repr', False, 'display.max_colwidth', 20, 'display.float_format', '{:.4f}'.format):
    display(merged_all)
#okay, and now I'll explode the descriptions, and everything from then on I can do grouped by title.

In [ ]:
merged_all["description"] = [[row[f"description_dfcdn"], row[f"description_edu"]] for num, row in merged_all.iterrows()]
merged_all = merged_all.drop(columns=[f"description_dfcdn", f"description_edu"])
merged_all = merged_all.explode("description")
merged_all = merged_all.drop_duplicates()
merged_all

In [ ]:
# So now I want to group-by-title and add as much information as possible. For that:
# * coverage an die description anhängen wenn vorhanden
# * die mit komplett leeren descriptions löschen
# * smart mergen:
#     * bei einigen ist nur die Jahreszahl anders, bei einigen steht halt nur "online" dabei, etc 
#     * sent_tokenize und einfach jeden satz 1 mal übernehmen?
#     * auf jeden Fall ungleiches concatenaten, und dann pro kurs die maximale nichtdoppelte beschreibung haben
# * merge with the siddata2021 dataset at least because I need the course number from there
#    * doch die origin-column behalten, vielleicht erkenn ich da die originalen siddata2021 kurse dran

In [ ]:
merged_all[merged_all["coverage"]=="Wed 30.0920 "] = pd.NA
merged_all["coverage"] = merged_all["coverage"].str.replace("Wed 30.0920 ", "").str.replace("Wed 31.0321 ", "")
merged_all["creator_name"] = [[j.get("title") for j in json.loads(i) if isinstance(j, dict)]  if isinstance(i, str) and i != "{}" else pd.NA for i in merged_all["creator"]]
merged_all["creator_jobtitle"] = [[j.get("job_title") for j in json.loads(i) if isinstance(j, dict)]  if isinstance(i, str) and i != "{}" else pd.NA for i in merged_all["creator"]]
merged_all = merged_all.drop(columns=["creator"])
merged_all = merged_all.dropna(how="all")
merged_all

In [ ]:
merged_all["creator_name"] = [tuple(i) if isinstance(i, list) else pd.NA for i in merged_all["creator_name"]]
merged_all["creator_jobtitle"] = [tuple(i) if isinstance(i, list) else pd.NA  for i in merged_all["creator_jobtitle"]]

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist] 

new = []
for num, (title, grouped) in enumerate(merged_all.groupby("title")):
    tmp = {}
#     print(title)
#     display(grouped)
    tmp["title"] = title
    for col in ["ddc_code", "format", "language", "publisher", "source", "type"]:
        tmp[col] = list(set(i for i in flatten([i for i in grouped[col]]) if not pd.isna(i)))
    for col in ["coverage", "start_semester", "url", "creator_name", "creator_jobtitle", "description"]:
        tmp[col] = list(set(i for i in grouped[col] if not pd.isna(i)))
    
    new.append(tmp)
        
new = pd.DataFrame(new)

In [ ]:
def check_subsumed(course):
    rm_inds = []
    descs = [nltk.sent_tokenize(part) for part in course]
    for ndesc, desc in enumerate(descs):
        for ndesc2, desc2 in enumerate(descs):
            if ndesc == ndesc2:
                continue
            if all(sent in desc for sent in desc2):
                if ndesc not in rm_inds:
                    rm_inds.append(ndesc2)
    return [i for n, i in enumerate(course) if n not in rm_inds]
 
    
def check_levenshtein(course):
    rm_inds = []
    for ndesc, desc in enumerate(course):
        for ndesc2, desc2 in enumerate(course):
            if ndesc == ndesc2:
                continue
            if distance(desc, desc2) / len(desc) < 0.003:
                if ndesc not in rm_inds:
                    rm_inds.append(ndesc2)
    return [i for n, i in enumerate(course) if n not in rm_inds]


def check_doubledot(course): #there are SO MANY courses where the only difference in the descriptions are stupid ".." instead of "." at sentence end
    return [desc.strip().replace("...", "<TRIPLEDOT>").replace("..", ".").replace("<TRIPLEDOT>", "...") for desc in course]

In [ ]:
#new["desc_len"] = [len([j for j in i if len(j) > 1]) for i in new["description"]]
new["veranstaltungsnummer"] = pd.NA
new["subtitle"] = pd.NA
new["title"] = new["title"].str.strip()
assert len(new[new.duplicated(subset=["title"])]) == 0
assert set(type(i) for i in new["description"]) == set([list])
print(len(new))
new.head()

## Merging with the old one

In [ ]:
orig = pd.read_csv("/home/chris/Documents/UNI_neu/Masterarbeit/data_new/siddata/raw_descriptions.csv")
orig["Name"] = orig["Name"].str.strip()
orig["Beschreibung"] = orig["Beschreibung"].str.strip()
orig = orig.rename(columns=dict(VeranstaltungsNummer="veranstaltungsnummer", Name="title", Untertitel="subtitle", Beschreibung="description"))
orig["description"] = orig["description"].str.strip().str.replace("\r\n", "\n")
orig["description"] = orig["description"].str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n")
orig["description"] = orig["description"].str.replace("\n", " ")
orig["description"] = orig["description"].str.strip()
orig["description"] = orig["description"].str.replace(r'<.*?>', '', regex=True)

In [ ]:
orig = orig.drop_duplicates()
# orig_singles = orig[~orig.duplicated(subset=["title"], keep=False)]
# orig_multis = orig[orig.duplicated(subset=["title"], keep=False)].drop_duplicates()

new_orig = []
for num, (title, grouped) in enumerate(orig.groupby("title")):
    tmp = {}
#     print(title)
#     display(grouped)
    tmp["title"] = title
    for col in ["veranstaltungsnummer", "subtitle", "description"]:
        tmp[col] = list(set(i for i in grouped[col] if not pd.isna(i)))
    new_orig.append(tmp)
        
new_orig = pd.DataFrame(new_orig)
new_orig

In [ ]:
alls = pd.concat([new, new_orig])
alls = alls.sort_values("title")
alls_singles = alls[~alls.duplicated(subset=["title"], keep=False)]
alls_multis = alls[alls.duplicated(subset=["title"], keep=False)]

In [ ]:
final_alls = []
for num, (title, grouped) in enumerate(alls_multis.groupby("title")):
    tmp = {}
#     print(title)
#     display(grouped)
    tmp["title"] = title
    for col in ["ddc_code", "format", "language", "publisher", "source", "type", "coverage", "start_semester", "url", "creator_name", "creator_jobtitle", "veranstaltungsnummer", "subtitle", "description"]:
        tmp[col] = [i for j in grouped[col] for i in (j if isinstance(j, list) else [j]) if not pd.isna(i)]
    final_alls.append(tmp)
        
final_alls = pd.DataFrame(final_alls)
final_alls = pd.concat([alls_singles, final_alls])
final_alls

In [ ]:
final_alls["n_descs"] = [len([j for j in i if len(j) > 1]) for i in final_alls["description"]]
assert all(isinstance(i, list) for i in final_alls["description"])
print(dict(Counter(final_alls["n_descs"])))

In [ ]:
final_alls["description"] = [list(set(desc)) for desc in final_alls["description"]]
print(dict(Counter(final_alls["n_descs"])))

In [ ]:
final_alls

In [ ]:
final_alls.to_csv("/home/chris/Documents/UNI_neu/Masterarbeit/data_new/siddata2022.csv")

In [ ]:
raise Exception()
#TODO I removed the stuff below argh -.-

In [ ]:
finals = new[new["desc_len"] == 1]
doubles = new[new["desc_len"] > 1].drop(columns=["desc_len"])
doubles.head()

#Ok, now we need to to the smart joining of the descriptions of doubles

In [ ]:
# inc_doubles = {}
# concat_descs = list(doubles["description"])
# for ncourse, course in enumerate(concat_descs):
#     course = check_doubledot(course)
#     course = check_subsumed(course)
#     course = check_levenshtein(course)
#     if len(course) == 1:
#         inc_doubles[ncourse] = course[0]
#     elif ncourse in SPECIALS:
#         pass
#     else:
#         print(f"{ncourse}/{len(concat_descs)}")
#         display(course)
        
#         descs = [nltk.sent_tokenize(part) for part in course]
#         descs = list({k: None for k in flatten(descs)}.keys())
#         merged = " ".join(descs)
#         print("merged: ", merged)
        
#         dowhat = input("[c]oncatenate? [m]erge?")
#         SPECIALS[ncourse] = dowhat

In [ ]:
#TODO now I could do some smarter stuff, like
# * If only one word is different, just add that word
# * Ignore special characters or just make them equal
# ..but, whatever.
# COULD ALSO 
# * When displaying, display in different color depending on which description it originally came from
# * Find if somebody else had this problem already and developed something smarter than me

In [ ]:
def merge_course_descs(course):
    course = check_doubledot(course)
    course = check_subsumed(course)
    course = check_levenshtein(course)
    if len(course) == 1:
        return course[0]
    else:
        descs = [nltk.sent_tokenize(part) for part in course]
        descs = list({k: None for k in flatten(descs)}.keys())
        return " ".join(descs)

In [ ]:
inc_doubles = {}
concat_descs = list(doubles["description"])
for ncourse, course in enumerate(tqdm(concat_descs)):
    inc_doubles[ncourse] = merge_course_descs(course)

In [ ]:
assert len(inc_doubles) == len(doubles)
doubles["description"] = inc_doubles.values()

In [ ]:
finals["description"] = [i[0] for i in finals["description"]]
finals = pd.concat([finals, doubles])
finals = finals.drop(columns=["desc_len"])

In [ ]:
finals.to_csv("/home/chris/Documents/UNI_neu/Masterarbeit/data_new/tmp.csv")

In [ ]:
from copy import deepcopy
fbkp = deepcopy(finals)

# Merging with the old one

In [ ]:
assert len(fbkp[fbkp.duplicated(subset=["title"])]) == 0

In [ ]:
finals = deepcopy(fbkp)

In [ ]:
# LEFT TO DO:
# * merge with the siddata2021 dataset at least because I need the course number from there
#    * doch die origin-column behalten, vielleicht erkenn ich da die originalen siddata2021 kurse dran

In [ ]:
finals["veranstaltungsnummer"] = pd.NA
finals["subtitle"] = pd.NA
finals["unchanged_2021"] = pd.NA

In [ ]:
orig.head()

In [ ]:
len(orig), len(orig["Name"].unique())

In [ ]:
finals = deepcopy(fbkp)
additionals = []
for num, (name, grouped) in enumerate(tqdm(orig.groupby("Name"))):
    handled = False
    grouped = grouped.drop_duplicates()
    unchanged2021 = True
    if len(grouped) > 1:
        has_desc = [i for i in grouped["Beschreibung"] if i and not pd.isna(i)]
        if len(has_desc) > 1:
            has_desc = merge_course_descs(has_desc)
            unchanged2021 = False
        merger_fn = lambda x: list({i:None for i in x.tolist() if i and not pd.isna(i)}.keys())
        grouped = grouped.agg(dict(VeranstaltungsNummer=merger_fn, Untertitel=merger_fn)).to_frame().T
        grouped["Name"] = name
        grouped["Beschreibung"] = has_desc[0] if len(has_desc) > 0 else pd.NA
    if len(grouped) == 1:
        corresp = finals[finals["title"] == name]
        if len(corresp) > 0:
            origtxt = list(grouped["Beschreibung"])[0]
            origtxt = origtxt.strip() if isinstance(origtxt, str) else ""
            newtxt = list(corresp["description"])[0]
            newtxt = newtxt.strip() if isinstance(newtxt, str) else ""
            row_ind = finals[finals["title"] == name].index; assert len(row_ind) == 1
            if not origtxt or origtxt == newtxt or distance(origtxt, newtxt) / len(origtxt) < 0.003:
#                 print("(almost) equal!")
                unchanged2021 = unchanged2021 or True
                handled = True
            else:
#                 print("merging!")
                unchanged2021 = False
                merged = merge_course_descs([origtxt, newtxt])
                finals.loc[row_ind[0], "description"] = merged
                handled = True
            if handled:
                finals.loc[row_ind[0], "subtitle"] = list(grouped["Untertitel"])[0]
                finals.loc[row_ind[0], "veranstaltungsnummer"] = list(grouped["VeranstaltungsNummer"])[0]
                finals.loc[row_ind[0], "unchanged_2021"] = unchanged2021
        else:  #do what? Add it? Add it only if it has a description? Assert that it doesn't have a description (I assume orig is a subset after all!)
            tmp = pd.DataFrame({"title": grouped["Name"], "veranstaltungsnummer": grouped["VeranstaltungsNummer"], "subtitle": grouped["Untertitel"], "description": grouped["Beschreibung"], "unchanged_2021": unchanged2021})
            additionals.append(tmp)
            handled = True            
    if not handled:
        display(grouped)
        break

In [ ]:
assert len(finals[finals.duplicated(subset=["title"])]) == 0

In [ ]:
additionals = pd.concat(additionals)

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', 25, 'display.expand_frame_repr', False, 'display.max_colwidth', 400, 'display.float_format', '{:.4f}'.format):
    display(additionals[additionals["unchanged_2021"]])

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', 25, 'display.expand_frame_repr', False, 'display.max_colwidth', 400, 'display.float_format', '{:.4f}'.format):
    display(finals[finals["unchanged_2021"] == True])

In [ ]:
finals = pd.concat([finals, additionals])

In [ ]:
finals.to_csv("/home/chris/Documents/UNI_neu/Masterarbeit/data_new/siddata2022.csv")

In [ ]:
#TODO: Rather than merging here, I can make the merging in the preprocessing - if I do that I can decide if I want to do the merging I'm doing here or just take the bag-of-words
#      -> another way to handle duplicates is make a relative bag-of-words: if a course exists 5 times, and one word comes up in all 5 descriptions, it get's a bow-value of 1, and if it occurs only in one it has 1/5th
#      -> I could do both if I just had a list of descriptions per name and do the processing later

In [ ]:
lans = get_langs(orig["description"].dropna(), assert_len=False)
orig["detected_lang"] = [lans[i] if not pd.isna(i) else None for i in orig["description"]]

In [ ]:
new_orig = groupby_merge(orig.drop_duplicates())

filtered = filt_de(filt_len(new_orig)).drop_duplicates(subset="title")
print("Number of UNIQUE german courses with >= 80 words that have a veranstaltungsnummer:", len(filtered["veranstaltungsnummer"].dropna()))
print("#Merged entries per title:", dict(enumerate(new_orig["description"].apply(lambda x: len(x)).value_counts()[:5])), "...")
new_orig.head()

## And Merging orig and new.

In [ ]:
new_orig, new = create_cols(new_orig, new)
alls = groupby_merge(pd.concat([new, new_orig])).dropna(subset="description")    #merge 
alls = alls[alls["description"].apply(lambda x: not all(pd.isna(i) for i in x))] #filter those that have ANY existing description
alls = alls.reset_index().drop_duplicates(subset="title").drop(columns="index")
make_tuples(alls)

#### Correct is_uos and veranstaltungsnummer:

In [ ]:
#ensure that those that are not from the original dataset don't have a veranstaltungsnummer
onlynotorig = lambda df: df[df["is_uos"].apply(lambda x: all(pd.isna(i) for i in x))]
assert all(pd.isna(j) for i in onlynotorig(alls)["veranstaltungsnummer"].unique() for j in i)

#ensure that the colum "veranstaltungsnummer" only exists for uos-courses
aslist = alls[alls["is_uos"].apply(lambda x: len(set(x)) > 1)][["is_uos", "veranstaltungsnummer"]].values.tolist()
for lislist in aslist:
    isuos_to_num = set(list(zip(*lislist)))-{(pd.NA,pd.NA)}
    assert all(i[0] == True for i in isuos_to_num) #ensures that if the veranstaltungsnummer is not pd.NA, is_uos must be True
#     if len(isuos_to_num) > 1 and len(set(i[1].split(".")[0] for i in isuos_to_num)) > 1:
#         #this are the (UOS!) courses that are in more than 1 FB
#         print([i[1] for i in isuos_to_num])

#now that we know that the Veranstaltungsnummer is REALLY ONLY given for the UOS-courses, we can set is_uos to True if any is_uos
alls["is_uos"] = alls["is_uos"].apply(lambda x: any(i == True for i in x if not pd.isna(i)))
alls.loc[~alls["is_uos"], "veranstaltungsnummer"] = None

In [ ]:
print("Number of UNIQUE german courses with >= 80 words that have a veranstaltungsnummer:", len(filt_de(filt_len(alls[~pd.isna(alls["veranstaltungsnummer"])]))))

In [ ]:
alls = squeeze_cols(alls)
alls

In [ ]:
alls.to_csv("/home/chris/Documents/UNI_neu/Masterarbeit/data_new/siddata2022_again.csv")

In [ ]:
#TODO: Rather than merging here, I can make the merging in the preprocessing - if I do that I can decide if I want to do the merging I'm doing here or just take the bag-of-words
#      -> another way to handle duplicates is make a relative bag-of-words: if a course exists 5 times, and one word comes up in all 5 descriptions, it get's a bow-value of 1, and if it occurs only in one it has 1/5th
#      -> I could do both if I just had a list of descriptions per name and do the processing later